# Linear symbolic Land-atmosphere example 

Testing platform for the symbolic equation version of the qgs model

In [ ]:
import sys, os
sys.path.extend([os.path.abspath('../')])

In [ ]:
import numpy as np
import sympy as sy
import sparse as sp
import math

In [ ]:
from qgs.params.params import QgParams
from qgs.functions.tendencies import create_tendencies

In [ ]:
from qgs.inner_products.symbolic import AtmosphericSymbolicInnerProducts, GroundSymbolicInnerProducts
from qgs.tensors.qgtensor import QgsTensor, QgsTensorDynamicT, QgsTensorT4
from qgs.tensors.symbolic_qgtensor import SymbolicQgsTensor, SymbolicQgsTensorDynamicT
from qgs.tensors.symbolic_qgtensor import _symbolic_tensordot, _shift_dict_keys, _add_to_dict

In [ ]:
model_parameters = QgParams({'phi0_npi': np.deg2rad(50.)/np.pi, 'n':1.3 }, dynamic_T=False)

In [ ]:
model_parameters.set_atmospheric_channel_fourier_modes(2, 2, mode="symbolic")
# Mode truncation at the wavenumber 2 in the x and at the 
# wavenumber 4 in the y spatial coordinates for the ocean
model_parameters.set_ground_channel_fourier_modes(2, 2, mode="symbolic")

In [ ]:
# Changing (increasing) the orography depth
model_parameters.ground_params.set_orography(0.2, 1)
# Setting the parameters of the heat transfer from the soil
model_parameters.gotemperature_params.set_params({'gamma': 1.6e7, 'T0': 300})
model_parameters.atemperature_params.set_params({ 'hlambda':10, 'T0': 290})
# Setting atmospheric parameters
model_parameters.atmospheric_params.set_params({'sigma': 0.2, 'kd': 0.085, 'kdp': 0.02})

# Setting insolation 
model_parameters.gotemperature_params.set_params({})

In [ ]:
C_g = 300
model_parameters.atemperature_params.set_insolation(0.4*C_g , 0)

model_parameters.gotemperature_params.set_insolation(C_g , 0)

In [ ]:
%%time
# Takes ~1 mins
gnd_ip_stored = GroundSymbolicInnerProducts(model_parameters, stored=True, return_symbolic=True, make_substitution=False) # <- Can be turned off once saved

In [ ]:
gnd_ip_stored = GroundSymbolicInnerProducts(model_parameters, stored=True, return_symbolic=True, make_substitution=True) # <- Can be turned off once saved

## Outputting model equations

In [ ]:
from qgs.functions.symbolic_output import create_symbolic_equations, translate_equations, equation_as_function, format_equations

Calculating the functions and tensor

In [ ]:
model_parameters.ground_params.hk[0].symbol

In [ ]:
%%time
funcs, = create_symbolic_equations(model_parameters, continuation_variables=[model_parameters.atemperature_params.eps], language='python')

Convert the dictionary of functions to a lambdified function (not working with numba)
The input `remain_variables` specifies which variables not to substitute with values.

In [ ]:
print(funcs)

In [ ]:
from numba import njit

Below function has the **kwargs input removed as I cannot get this to work with numba

In [ ]:
@njit
def f(t, U):
	#Tendency function of the qgs model
	F = np.empty_like(U)
	C_go = 300
	C_a = 0.4 * C_go
	F[0] = -0.0425*U[0] + 0.0425*U[10] - 0.156054828133898*U[12] + 0.156054828133898*U[2]
	F[1] = -0.980418808722262*U[0]*U[2] - 0.980418808722262*U[10]*U[12] + 0.0425*U[11] - 1.56867009395562*U[13]*U[15] - 1.50055762081784*U[14]*U[17] + 1.50055762081784*U[15]*U[16] - 0.0425*U[1] + 0.101317695204044*U[2] - 1.56867009395562*U[3]*U[5] - 1.50055762081784*U[4]*U[7] + 1.50055762081784*U[5]*U[6]
	F[2] = 0.980418808722262*U[0]*U[1] - 0.0580129472616723*U[0] + 0.980418808722262*U[10]*U[11] + 0.0580129472616723*U[10] + 0.0425*U[12] + 1.56867009395562*U[13]*U[14] + 1.50055762081784*U[14]*U[16] + 1.50055762081784*U[15]*U[17] - 0.101317695204044*U[1] - 0.0425*U[2] + 1.56867009395562*U[3]*U[4] + 1.50055762081784*U[4]*U[6] + 1.50055762081784*U[5]*U[7]
	F[3] = 1.87265793760678*U[11]*U[15] - 1.87265793760678*U[12]*U[14] + 0.0425*U[13] - 0.0624219312535594*U[15] + 3.74531587521356*U[16]*U[19] - 3.74531587521356*U[17]*U[18] + 1.87265793760678*U[1]*U[5] - 1.87265793760678*U[2]*U[4] - 0.0425*U[3] + 0.0624219312535594*U[5] + 3.74531587521356*U[6]*U[9] - 3.74531587521356*U[7]*U[8]
	F[4] = -1.02902937637678*U[0]*U[5] - 1.02902937637678*U[10]*U[15] + 1.73752196836555*U[11]*U[17] + 0.574852231579351*U[12]*U[13] - 1.73752196836555*U[12]*U[16] + 0.0425*U[14] - 0.0342706502636204*U[17] + 1.73752196836555*U[1]*U[7] + 0.574852231579351*U[2]*U[3] - 1.73752196836555*U[2]*U[6] - 0.0425*U[4] + 0.0478988752370612*U[5] + 0.0342706502636204*U[7]
	F[5] = 1.02902937637678*U[0]*U[4] + 1.02902937637678*U[10]*U[14] - 0.574852231579351*U[11]*U[13] - 1.73752196836555*U[11]*U[16] - 1.73752196836555*U[12]*U[17] + 0.0438818497388818*U[13] + 0.0425*U[15] + 0.0342706502636204*U[16] - 0.574852231579351*U[1]*U[3] - 1.73752196836555*U[1]*U[6] - 1.73752196836555*U[2]*U[7] - 0.0438818497388818*U[3] - 0.0478988752370612*U[4] - 0.0425*U[5] - 0.0342706502636204*U[6]
	F[6] = -2.71889339738442*U[0]*U[7] - 2.71889339738442*U[10]*U[17] + 0.753865979381443*U[11]*U[15] + 0.753865979381443*U[12]*U[14] - 4.35022943581506*U[13]*U[19] - 0.0251288659793814*U[15] + 0.0425*U[16] + 0.753865979381443*U[1]*U[5] + 0.753865979381443*U[2]*U[4] - 4.35022943581506*U[3]*U[9] + 0.0251288659793814*U[5] - 0.0425*U[6] + 0.0702434536337315*U[7]
	F[7] = 2.71889339738442*U[0]*U[6] + 2.71889339738442*U[10]*U[16] - 0.753865979381443*U[11]*U[14] + 0.753865979381443*U[12]*U[15] + 4.35022943581506*U[13]*U[18] + 0.0251288659793814*U[14] + 0.0425*U[17] - 0.753865979381443*U[1]*U[4] + 0.753865979381443*U[2]*U[5] + 4.35022943581506*U[3]*U[8] - 0.0251288659793814*U[4] - 0.0702434536337315*U[6] - 0.0425*U[7]
	F[8] = -2.26482546109569*U[0]*U[9] - 2.26482546109569*U[10]*U[19] - 1.7450294536311*U[13]*U[17] + 0.0425*U[18] - 1.7450294536311*U[3]*U[7] - 0.0425*U[8] + 0.050658847602022*U[9]
	F[9] = 2.26482546109569*U[0]*U[8] + 2.26482546109569*U[10]*U[18] + 1.7450294536311*U[13]*U[16] + 0.0425*U[19] + 1.7450294536311*U[3]*U[6] - 0.050658847602022*U[8] - 0.0425*U[9]
	F[10] = 4.68670500616653e-6*C_a + 0.00386363636363636*U[0] - 0.023715438957012*U[10] - 1.41868025576271*U[11]*U[2] + 1.41868025576271*U[12]*U[1] + 0.0141868025576271*U[12] - 1.13494420461017*U[14]*U[5] + 1.13494420461017*U[15]*U[4] - 2.83736051152543*U[16]*U[7] + 2.83736051152543*U[17]*U[6] - 2.26988840922034*U[18]*U[9] + 2.26988840922034*U[19]*U[8] + 0.00645434108527132*U[20] - 0.0141868025576271*U[2]
	F[11] = -1.43757363347933*U[0]*U[12] + 1.02191932371371*U[10]*U[2] - 0.0315441157231782*U[11] + 0.0214771158470353*U[12] + 1.63507091794193*U[13]*U[5] + 1.21855791962175*U[14]*U[7] - 2.30011781356693*U[15]*U[3] - 1.21855791962175*U[15]*U[6] + 1.85472813238771*U[16]*U[5] - 1.85472813238771*U[17]*U[4] + 0.00900906225374311*U[1] + 0.00559477950653936*U[21]
	F[12] = 1.43757363347933*U[0]*U[11] + 0.0122974647859652*U[0] - 1.02191932371371*U[10]*U[1] - 0.0122974647859652*U[10] - 0.0214771158470353*U[11] - 0.0315441157231782*U[12] - 1.63507091794193*U[13]*U[4] + 2.30011781356693*U[14]*U[3] - 1.21855791962175*U[14]*U[6] - 1.21855791962175*U[15]*U[7] + 1.85472813238771*U[16]*U[4] + 1.85472813238771*U[17]*U[5] + 0.00559477950653936*U[22] + 0.00900906225374311*U[2]
	F[13] = -1.24843862507119*U[11]*U[5] + 1.24843862507119*U[12]*U[4] - 0.0363121306090808*U[13] - 2.3185288751322*U[14]*U[2] + 2.3185288751322*U[15]*U[1] + 0.017834837501017*U[15] - 2.49687725014237*U[16]*U[9] + 2.49687725014237*U[17]*U[8] - 4.63705775026441*U[18]*U[7] + 4.63705775026441*U[19]*U[6] + 0.00507126799557032*U[23] + 0.0121428571428571*U[3] - 0.017834837501017*U[5]
	F[14] = -1.16886956037576*U[0]*U[15] + 0.422511733532696*U[10]*U[5] - 0.612715105162524*U[11]*U[7] - 1.38291034440645*U[12]*U[3] + 0.612715105162524*U[12]*U[6] + 1.79985224341047*U[13]*U[2] - 0.0412871146288803*U[14] + 0.0173705927405276*U[15] - 1.87294455066922*U[16]*U[2] + 1.87294455066922*U[17]*U[1] + 0.0124282982791587*U[17] + 0.00452503199094866*U[24] + 0.015412683237731*U[4] - 0.0124282982791587*U[7]
	F[15] = 1.16886956037576*U[0]*U[14] - 0.422511733532696*U[10]*U[4] + 1.38291034440645*U[11]*U[3] + 0.612715105162524*U[11]*U[6] + 0.612715105162524*U[12]*U[7] - 1.79985224341047*U[13]*U[1] - 0.0159138129390846*U[13] - 0.0173705927405276*U[14] - 0.0412871146288803*U[15] - 1.87294455066922*U[16]*U[1] - 0.0124282982791587*U[16] - 1.87294455066922*U[17]*U[2] + 0.00452503199094866*U[25] + 0.0159138129390846*U[3] + 0.015412683237731*U[5] + 0.0124282982791587*U[6]
	F[16] = -2.94535914360826*U[0]*U[17] + 0.569389237785845*U[10]*U[7] - 0.768581081081081*U[11]*U[5] - 0.768581081081081*U[12]*U[4] + 0.911022780457352*U[13]*U[9] + 1.42736486486486*U[14]*U[2] + 1.42736486486486*U[15]*U[1] + 0.0109797297297297*U[15] - 0.0460906434981493*U[16] + 0.0306919594705944*U[17] - 4.71257462977322*U[19]*U[3] + 0.00399762116767931*U[26] - 0.0109797297297297*U[5] + 0.0185698198198198*U[6]
	F[17] = 2.94535914360826*U[0]*U[16] - 0.569389237785845*U[10]*U[6] + 0.768581081081081*U[11]*U[4] - 0.768581081081081*U[12]*U[5] - 0.911022780457352*U[13]*U[8] - 1.42736486486486*U[14]*U[1] - 0.0109797297297297*U[14] + 1.42736486486486*U[15]*U[2] - 0.0306919594705944*U[16] - 0.0460906434981493*U[17] + 4.71257462977322*U[18]*U[3] + 0.00399762116767931*U[27] + 0.0109797297297297*U[4] + 0.0185698198198198*U[7]
	F[18] = -2.37660377951895*U[0]*U[19] + 0.0288656329496225*U[10]*U[9] + 1.50101291338039*U[13]*U[7] - 3.30992591155675*U[17]*U[3] - 0.0513521112007289*U[18] + 0.026256705211838*U[19] + 0.00341993024749443*U[28] + 0.0220279383429672*U[8]
	F[19] = 2.37660377951895*U[0]*U[18] - 0.0288656329496225*U[10]*U[8] - 1.50101291338039*U[13]*U[6] + 3.30992591155675*U[16]*U[3] - 0.026256705211838*U[18] - 0.0513521112007289*U[19] + 0.00341993024749443*U[29] + 0.0220279383429672*U[9]
	F[20] = 6.44421938347898e-6*C_go + 0.0172043158333333*U[10] - 0.00976477713178294*U[20]
	F[21] = 0.0172043158333333*U[11] - 0.00976477713178294*U[21]
	F[22] = 0.0172043158333333*U[12] - 0.00976477713178294*U[22]
	F[23] = 0.0172043158333333*U[13] - 0.00976477713178294*U[23]
	F[24] = 0.0172043158333333*U[14] - 0.00976477713178294*U[24]
	F[25] = 0.0172043158333333*U[15] - 0.00976477713178294*U[25]
	F[26] = 0.0172043158333333*U[16] - 0.00976477713178294*U[26]
	F[27] = 0.0172043158333333*U[17] - 0.00976477713178294*U[27]
	F[28] = 0.0172043158333333*U[18] - 0.00976477713178294*U[28]
	F[29] = 0.0172043158333333*U[19] - 0.00976477713178294*U[29]
	return F

In [ ]:
offset = 1 if model_parameters.dynamic_T else 0

In [ ]:
model_parameters.ndim

## Comparing with numerical results

In [ ]:
from qgs.integrators.integrator import RungeKuttaIntegrator, RungeKuttaTglsIntegrator
import matplotlib.pyplot as plt

In [ ]:
f_num, Df, ten_num = create_tendencies(model_parameters, return_qgtensor=True)

In [ ]:
integrator = RungeKuttaIntegrator()
integrator.set_func(f)

In [ ]:
integrator_num = RungeKuttaIntegrator()
integrator_num.set_func(f_num)

In [ ]:
# ICs calculated from long transient

ic = np.array([0.05055959, -0.01639403, -0.01440781, -0.01846523, -0.01352099,
        0.011685  , -0.00201673, -0.02030682,  0.03923588, -0.02229535,
        0.0586372 , -0.01805569, -0.01264252, -0.0103574 , -0.00618456,
        0.01159318, -0.00478694, -0.00782509,  0.01066059, -0.01552667,
        0.30718325, -0.03247899, -0.04512935, -0.00078786, -0.00067468,
        0.00183836,  0.00068025,  0.00215424, -0.00322845, -0.00186392])

In [ ]:
%%time
integrator.integrate(0., 100000., 0.1, ic=ic, write_steps=5)
reference_time, reference_traj = integrator.get_trajectories()

In [ ]:
%%time
integrator_num.integrate(0., 100000., 0.1, ic=ic, write_steps=5)
reference_time_num, reference_traj_num = integrator_num.get_trajectories()

In [ ]:
varx = 2
vary = 1
plt.figure(figsize=(12, 10))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')
plt.plot(reference_traj_num[varx], reference_traj_num[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
varx = 2
vary = 1
plt.figure(figsize=(12, 10))

plt.plot(reference_traj[varx], reference_traj[vary], marker='o', ms=0.07, ls='')
plt.plot(reference_traj_num[varx], reference_traj_num[vary], marker='o', ms=0.07, ls='')

plt.xlabel('$'+model_parameters.latex_var_string[varx]+'$')
plt.ylabel('$'+model_parameters.latex_var_string[vary]+'$');

In [ ]:
plt.figure(figsize=(12, 8))
t_s = 100000

plt.plot(reference_time_num[:t_s] * model_parameters.dimensional_time, reference_traj_num[varx, :t_s])
plt.plot(reference_time[:t_s] * model_parameters.dimensional_time, reference_traj[varx, :t_s])
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
t_s = 1000

vars = [1, 2]

plt.plot(reference_traj_num[vars, :-1].T)
# plt.plot(reference_time[:t_s] * model_parameters.dimensional_time, reference_traj[varx, :t_s])
plt.show()